# Another approach to f.d

alternative way of computing dg

In [ ]:
# f_linear = torch.nn.Linear(2,784)
#psi = f_linear(tgrid)
psi = decoder(tgrid)

hx = 1/numsteps
hy = 1/numsteps
dpsidx = diff_by_x(psi, hx)
dpsidy = diff_by_y(psi, numsteps, hy)

dpsidx_second = diff_by_x(dpsidx, hx)

dpsidx_dy = diff_by_y(dpsidx, numsteps, hy)
dpsidy_second = diff_by_y(dpsidy, numsteps, hy)

NameError: name 'diff_by_x' is not defined

In [ ]:
dot_vmap = torch.func.vmap(torch.dot)
dg_11_dx = 2 * dot_vmap(dpsidx_second, dpsidx)
dg_12_dx = dot_vmap(dpsidx_second, dpsidy) + dot_vmap(dpsidx_dy, dpsidx)
dg_22_dx = 2 * dot_vmap(dpsidx_dy, dpsidy)

dg_dx = torch.cat( (dg_11_dx.unsqueeze(1), dg_12_dx.unsqueeze(1),
                    dg_12_dx.unsqueeze(1), dg_22_dx.unsqueeze(1)), dim=1) 
dg_dx = dg_dx.reshape(numsteps*numsteps, 2, 2)

Verification

In [ ]:
dg_dx[202]

In [ ]:
Frob_dg = dg_jacfwd.reshape(numsteps*numsteps, -1).norm(dim=-1).view(numsteps,numsteps)

# Create the ticks for the axes (3 ticks between the borders)
num_ticks = 5  # 3 between borders + 2 borders
ticks = np.linspace(-size/2, size/2, num_ticks)
tick_labels = [f'{tick:.2f}' for tick in ticks]  # Formatting tick labels for clarity

# Plotting the colormap
plt.figure(figsize=(6, 6))  # Square figure
plt.imshow(Frob_dg.detach(), cmap='viridis', origin='lower', extent=[-size/2, size/2, -size/2, size/2])
plt.colorbar(label='Frobenius norm of dg', shrink = 0.7)

# Set ticks and labels for both axes
plt.xticks(ticks=ticks, labels=tick_labels)
plt.yticks(ticks=ticks, labels=tick_labels)

# Set axis labels and title
plt.title('Finite differences: Frobenius norm of metric on a grid')
plt.xlabel('X Axis')
plt.ylabel('Y Axis')

plt.grid(False)  # Disable grid lines on the plot
plt.show()


In [ ]:
dg_dx_fd_no_borders = dg_dx_fd.reshape(numsteps, numsteps, -1)[2:-2,2:-2]
dg_dx_fd_no_borders = dg_dx_fd_no_borders.reshape(-1, 2, 2)
dg_dx_fd_no_borders_norms = dg_dx_fd_no_borders.norm(dim=(1,2)).detach()

In [ ]:
plt.hist(dg_dx_fd_no_borders_norms, bins = 30)
plt.show

In [ ]:
## This is old 
# derivatives of the metric on a grid derived from d\Psi
def dg_grid (grid): #dg
    
    numsteps = int(np.sqrt(grid.shape[0]))
    
    hx = float(abs((grid[numsteps**2 - 1] - grid[0])[0]))/(numsteps - 1)
    hy = float(abs((grid[numsteps**2 - 1] - grid[0])[1]))/(numsteps - 1)

    latent = grid
    #latent = latent.to(device)
    psi = decoder(latent)
    
    dpsidx = diff_by_x(psi, hx)
    dpsidy = diff_by_y(psi, numsteps, hy)

    dpsidx_second = diff_by_x(dpsidx, hx)
    
    dpsidx_dy = diff_by_y(dpsidx, numsteps, hy)
    dpsidy_second = diff_by_y(dpsidy, numsteps, hy)
    
    

    return metric_der
    """
    dgdx = torch.cat((2*(dpsidx*dpsidx_second).sum((1,2,3)),(dpsidx_second * dpsidy + dpsidx * dpsidx_dy).sum((1,2,3)),
                      (dpsidx_second * dpsidy + dpsidx * dpsidx_dy).sum((1,2,3)),2*(dpsidy * dpsidx_dy).sum((1,2,3))),0)
    dgdy = torch.cat((2*(dpsidx*dpsidx_dy).sum((1,2,3)),(dpsidy_second * dpsidx + dpsidy * dpsidx_dy).sum((1,2,3)),
                      (dpsidy_second * dpsidx + dpsidy * dpsidx_dy).sum((1,2,3)),2*(dpsidy*dpsidy_second).sum((1,2,3))),0)
    metric_der = torch.cat((dgdx, dgdy), 0)
    metric = metric_der
    
    metric = metric.view(8, numsteps*numsteps)
    metric = metric.transpose(0, 1)
    metric = metric.view(numsteps*numsteps, 2, 4)
    metric = metric.view(numsteps*numsteps, 2, 2, 2)
    
    return metric
    """

In [ ]:
# compute the grid of metric derivatives
with torch.no_grad():
    metric_der = dg_grid(tgrid)
    #metric_der = specific_metric_der(tgrid)


In [ ]:
metric_der.shape

In [ ]:
#This means that we can simultanuousely invert all the matrices over the grid
torch.equal(torch.inverse(metric[0]),torch.inverse(metric)[0])

In [ ]:
#this is the inverse of the metric on a grid
metric_inv = torch.inverse(metric)

# Christoffel symbols 

In [ ]:
#Christoffel symbols on a grid
def Ch_grid(grid, metric_inv=metric_inv, metric_der=metric_der):
    #x = grid[:,0]
    #y = grid[:, 1]
    n = grid.shape[0]
    Ch = torch.zeros((n, 2,2,2))
    for i in range(2):
        for j in range(2):
            for l in range(2):
                for k in range(2):
                    #Ch^l_ij
                    Ch[:,l,i,j] += 0.5 * metric_inv[:,l,k] * (metric_der[:,i,k,j] + metric_der[:,j,i,k] - metric_der[:,k,i,j]) 
                    
                    #Ch[l,i,j] += 0.5 * g_inv(grid)[l,k] * (dg(grid)[i,k,j] + dg(grid)[j,i,k] - dg(grid)[k,i,j]) #Ch^l_ij
    return Ch

In [ ]:
#checking Christoffel on a grid
Ch_grid(tgrid).shape

Derivatives of Christoffel symbols on a grid

In [ ]:
#derivatives of Christoffel symbols on a grid
def Ch_der_grid(grid,metric_inv=metric_inv, metric_der=metric_der):
    n = grid.shape[0]

    numsteps = int(np.sqrt(grid.shape[0]))
    hx = float(abs((grid[numsteps**2 - 1] - grid[0])[0]))/(numsteps - 1)
    hy = float(abs((grid[numsteps**2 - 1] - grid[0])[1]))/(numsteps - 1)
    
    Chdx = diff_by_x(Ch_grid(grid,metric_inv, metric_der), numsteps, hx)
    Chdy = diff_by_y(Ch_grid(grid,metric_inv, metric_der), numsteps, hy)
    Chder = torch.cat((Chdx, Chdy), -1)
    Chder = Chder.view(n,2,2,2,2)
    Chder = Chder.transpose(-1,-2)
    return Chder



In [ ]:
Ch_der_grid(tgrid).shape

In [ ]:
# Riemann curvature tensor (3,1)
def Riem_old(grid,metric_inv=metric_inv, metric_der=metric_der):
    n = grid.shape[0]
    Ch = Ch_grid(grid,metric_inv, metric_der)
    Ch_der = Ch_der_grid(grid,metric_inv, metric_der)

    Riem = torch.zeros(n, 2, 2, 2, 2)
    for i in range(2):
        for j in range(2):
            for k in range(2):
                for l in range(2):                    
                    Riem[:, i, j, k, l] = Ch_der[:, i, l, j, k] - Ch_der[:, i, k, j, l] 
                    for p in range(2):
                        Riem[:, i, j, k, l] += (Ch[:, i, k, p]*Ch[:, p, l, j] - Ch[:, i, l, p]*Ch[:, p, k, j])
    return Riem



In [ ]:
# Riemann curvature tensor (3,1)
def Riem(grid,metric_inv=metric_inv, metric_der=metric_der):
    Ch = Ch_grid(grid,metric_inv, metric_der)
    Ch_der = Ch_der_grid(grid,metric_inv, metric_der)

    Riem = torch.einsum("tiljk->tijkl",Ch_der) - torch.einsum("tikjl->tijkl",Ch_der)
    Riem += torch.einsum("tikp,tplj->tijkl", Ch, Ch) - torch.einsum("tilp,tpkj->tijkl", Ch, Ch)
    return Riem


In [ ]:
Riem(tgrid)

In [ ]:
#Scew symmetry check
torch.equal(Riem(tgrid)[:,0,0,0,1], - Riem(tgrid)[:,0,0,1,0])


In [ ]:
Riem_old(tgrid)

In [ ]:
SlowRiemann = Riem_old(tgrid)
FastRiemann = Riem(tgrid)
torch.equal(SlowRiemann, FastRiemann)

In [ ]:
(abs(SlowRiemann-FastRiemann)).max()

In [ ]:
# Ricci curvature tensor via Riemann
# R_ab = Riem^c_acb
def Ric_old(grid,metric_inv=metric_inv, metric_der=metric_der):
    n = grid.shape[0]
    Ric = torch.zeros(n, 2, 2)
    Riemann = Riem(grid,metric_inv, metric_der)

    for a in range(2):
        for b in range(2):
            for c in range(2):
                Ric[:, a, b] += Riemann[:, c, a, c, b]
    return Ric
    # takes 2.5 secs on 100 by 100 grid

In [ ]:
def Ric(grid,metric_inv=metric_inv, metric_der=metric_der):
    Riemann = Riem(grid,metric_inv, metric_der)
    Ric = torch.einsum("tcacb->tab",Riemann)
    return Ric

In [ ]:
FastRicci = Ric(tgrid)

In [ ]:
SlowRicci = Ric_old(tgrid)

In [ ]:
torch.equal(SlowRicci, FastRicci)

In [ ]:
# Scalar curvature tensor via Riemann and Ricci
# R_ab = Riem^c_acb
# R = g^ij * R_ij
def Sc_old(grid,metric_inv=metric_inv, metric_der=metric_der):
    Riemann = Riem_old(grid,metric_inv, metric_der)
    n = grid.shape[0]
    Sc = torch.zeros(n)
    Ric = torch.zeros(n, 2, 2)
    
    for a in range(2):
        for b in range(2):
            for c in range(2):
                Ric[:, a, b] += Riemann[:, c, a, c, b]
    Sc = metric_inv*Ric
    Sc = torch.sum(Sc,(1,2))
    return Sc

In [ ]:
def Sc(grid,metric_inv=metric_inv, metric_der=metric_der):
    Ricci = Ric(grid,metric_inv, metric_der)
    #Sc = metric_inv * Ricci
    #Sc = torch.sum(Sc,(1,2))
    Sc = torch.einsum("tij,tij->t", metric_inv, Ricci)
    return Sc

In [ ]:
Scalar_curvature_grid = Sc(tgrid)

In [ ]:

Scalar_curvature_grid_old = Sc_old(tgrid)

In [ ]:
(abs(Scalar_curvature_grid_old-Scalar_curvature_grid)).max()

# Scalar curvature heatmap

In [ ]:
def make_Sc_curv_plot (grid = tgrid):
    Scalar_curvature_grid = Sc(tgrid)
    # Fast computation of Frobenious norm on the grid without borders
    Scalar_curv = Scalar_curvature_grid.view(numsteps,numsteps)
    #Scalar_curv_check = Scalar_curv[30:-30,30:-30].transpose(0,1)
    Scalar_curv = Scalar_curv[2:-2,2:-2].transpose(0,1)

    
    #Heat map of the Scalar curvature
    h = plt.contourf(xs[2:-2], ys[2:-2], Scalar_curv)
    #h = plt.contourf(xs[30:-30], ys[30:-30], Scalar_curv_check)
    plt.title('Heat map of the Scalar curvature ')
    plt.xlabel( "x coordinate")
    plt.ylabel( "y coordinate")
    plt.axis('scaled')
    #plt.xlim(-1.5,1.5)
    #plt.ylim(-1.5,1.5)

    plt.colorbar()
    plt.show()
    return

In [ ]:
make_Sc_curv_plot(tgrid)

Simplified energy functional computation: $F_{new}(g) = \int_{M}  R^{2} d\mu$

In [ ]:
def curv_func(metric = metric, grid = tgrid):
    metric_no_border = metric.reshape(numsteps, numsteps,2,2)[2:-2,2:-2]
    det_metric_no_border = torch.det(metric_no_border)
    det_sqrt = torch.sqrt(det_metric_no_border)
    #grid = tgrid
    Scalar_curvature_grid = Sc(grid)
    # Fast computation of Frobenious norm on the grid without borders
    Scalar_curv = Scalar_curvature_grid.view(numsteps,numsteps)
    #Scalar_curv_check = Scalar_curv[30:-30,30:-30].transpose(0,1)
    Scalar_curv = Scalar_curv[2:-2,2:-2].transpose(0,1)

    hx = float(abs((grid[numsteps**2 - 1] - grid[0])[0]))/(numsteps - 1)
    hy = float(abs((grid[numsteps**2 - 1] - grid[0])[1]))/(numsteps - 1)

    F_new = (det_sqrt*torch.square(Scalar_curv)*hx*hy).sum()
    print(F_new)
    return

curv_func(metric, tgrid)

In [ ]:
#no curvature ~240
#1 epoch with curvature ~66
#10 epochs with curvature ~56

# Geodesics

In [ ]:
# This is used for making a piecewise constant metric from its evaluation on a grid
def find_nearest_index (grid, u):
    index = int(torch.min(abs(grid - u),0).indices.sum())
    #index = int((((u - tgrid[0])*numsteps/size).floor()*torch.tensor([1.,numsteps])).sum()) #thisd could be faster
    return index

In [ ]:
#computing geodesics...
# y = [u , v]
# v := dot(u)
# dot(v)^l = Ch^l_ij * v^i * v^j
def geod(y, t):
    #u, v = y
    u = y[0:2:]
    v = y[2::]
    dudt = v
    #dvdt = torch.zeros(2)
    dvdt = np.zeros(2)
    u = torch.from_numpy(u)
    for l in range(2):
        for i in range(2):
            for j in range(2):
                dvdt[l] -= (Ch(u)[l,i,j]).numpy() * v[i] * v[j]
    dydt = np.concatenate((dudt, dvdt))
    #dydt = torch.cat((dudt, dvdt),0)
    return dydt

## Vectorized computation of geodesics (with a loop in find_indices)

In [ ]:
# this could be done faster
def find_nearest_indices (grid, u):
    #this could be done more efficiently
    n = u.shape[0]
    indices = torch.zeros(n)
    for i in range(n):
        indices[i] = find_nearest_index(grid, u[i])
    indices = indices.to(torch.int64) # just some magic to make it work
    return indices

In [ ]:
find_nearest_index(tgrid, torch.tensor([0.5,0.3]))

In [ ]:
#evaluation of the piecewise constant inverse of g
def g_inv_vect (grid, u): #inverse metric
    #index = find_nearest_index(tgrid, u)
    indices = find_nearest_indices(grid, u)
    #A = metric[index]
    A = torch.index_select(metric, 0, indices)
    g_inv = torch.inverse(A)
    return g_inv

In [ ]:
#g_inv_vect(tgrid, check)

In [ ]:
#evaluation of the piecewise constant derivatives of g
def dg_vect (grid, u): #dg
    #index = find_nearest_index(uniform_grid, u)
    indices = find_nearest_indices(grid, u)
    g = torch.index_select(metric_der, 0, indices)
    return g

In [ ]:
#dg_vect(tgrid, check)

In [ ]:
#Christoffel symbols at a vector of n points. u has shape (n, x, y)
def Ch_vect(grid, u):
    n = u.shape[0]
    Ch = torch.zeros((n,2,2,2))
    for i in range(2):
        for j in range(2):
            for l in range(2):
                for k in range(2):
                    Ch[:,l,i,j] += 0.5 * g_inv_vect(grid, u)[:,l,k] * (dg_vect(grid, u)[:,i,k,j] + dg_vect(grid, u)[:,j,i,k] - dg_vect(grid, u)[:,k,i,j]) #Ch^l_ij
    return Ch

In [ ]:
#Ch_vect(tgrid, check)

In [ ]:
#Ch(check[1])
# just to check there is no mistake in vectorized vertion Ch_vect

In [ ]:
#  Ch_vect still exploits the loop in find_indices
#Ch_vect(tgrid,tgrid)

In [ ]:
#computing geodesics...
# y has shape num of points, u, v
# v := dot(u)
# dot(v)^l = Ch^l_ij * v^i * v^j
def geod(y, t):
    #u, v = y
    n = y.shape[0]
    u = y[: , 0:2:]
    v = y[: , 2::]
    dudt = v
    dvdt = torch.zeros(n, 2)
    for l in range(2):
        for i in range(2):
            for j in range(2):
                dvdt[:, l] -= Ch_vect(tgrid, u)[:, l,i,j] * v[:, i] * v[:, j] #here we use Ch_vect instead od Ch
    dydt = torch.cat((dudt.T, dvdt.T)).T
    # dydt = np.concatenate((dudt, dvdt))
    return dydt

In [ ]:
def rungekutta_new(f, y0, t, args=()):
    nt = len(t) # number of steps in time
    # len(y0[0]) is the number of initial conditions
    # len(y0[1]) is the dimention of the state space. In our case it is 4 
    y = torch.zeros((nt, y0.shape[0],y0.shape[1]))
    y[0,:,:] = y0
    for i in range(nt - 1):
        y[i+1,:,:] = y[i,:,:] + (t[i+1] - t[i])*f(y[i,:,:], t[i], *args)
        print(y[i,:,:])
    return y

In [ ]:
zoom = 1

# Let us start at random points u with the same speed v
# we want to draw m geodesics
m = 10
v = torch.tensor([0.00, 0.00,1.00])/zoom
v = v.repeat(m,1)
u = torch.rand(m,1)/zoom
#unorm = u.norm(dim=1)
#u = (u.T/unorm).T

RandStartComSpeed = torch.cat((u,v),1)
RandStartComSpeed

In [ ]:
t = torch.linspace(0, 1, steps = 21)
sol3 = rungekutta_new(geod, RandStartComSpeed, t)

In [ ]:
plt.plot(sol3[:15, :, 0], sol3[:15, :, 1]) #geodesics are shortened by step 15 because of border effects
plt.title( "Plots of geodesics with rnd ititial point and common initial speed")
plt.xlabel( "x coordinate")
plt.ylabel( "y coordinate")
plt.grid()

# Scalar curvature and geodesics on one plot

In [ ]:
# Let us start at different initial points u with the same speed v
# we want to draw m geodesics
m = 15 #number of geodesics
#v = torch.tensor([0.00, 0.00,1.00])
v = torch.tensor([0.00, 0.00,1.00/zoom])
v = v.repeat(m,1)
#u = torch.rand(m,1)
#u = torch.linspace(0.01,1.51,steps=m).reshape(15,1)
u = torch.linspace(0.01,(m/10+0.01)/zoom,steps=m).reshape(15,1)
#unorm = u.norm(dim=1)
#u = (u.T/unorm).T

RandStartComSpeed2 = torch.cat((u,v),1)
RandStartComSpeed2

In [ ]:
t = torch.linspace(0, 1, steps = 41)
sol4 = rungekutta_new(geod, RandStartComSpeed2, t)

In [ ]:
#Scalar curvature and geodesics
h = plt.contourf(xs[2:-2], ys[2:-2], Scalar_curv)
plt.plot(sol4[:, :, 0], sol4[:, :, 1]) #geodesics are shortened by step 30 because of border effects
plt.title('Scalar curvature and geodesics')
plt.xlabel( "x coordinate")
plt.ylabel( "y coordinate")
plt.axis('scaled')
plt.xlim(0,1.75/zoom)
plt.ylim(0,1.25/zoom)
plt.colorbar(label="Scalar curvature")
plt.show()

# Evolution of the curvature under the discretization of the Ricci flow

#### $g_{i+1} = g_i - 2Ric \cdot dt$

In [ ]:
# g(t_i+1)
dt = 0.1 # size of one step
steps = 10 # number of evolution steps

In [ ]:
def metric_der_not_using_decoder(grid=tgrid,metric=metric):
    # compute step sizes
    grid = tgrid
    numsteps = int(np.sqrt(grid.shape[0]))
        
    hx = float(abs((grid[numsteps**2 - 1] - grid[0])[0]))/(numsteps - 1)
    hy = float(abs((grid[numsteps**2 - 1] - grid[0])[1]))/(numsteps - 1)

    dgdx = diff_by_x(metric, numsteps, hx)
    dgdy = diff_by_y(metric, numsteps, hy)

    metric_der = torch.cat((dgdx, dgdy), -1)
    metric_der = metric_der.view(numsteps*numsteps, 2, 2, 2)
    metric_der = metric_der.transpose(-1,-2)

    #metric_der = torch.cat((dgdx, dgdy), 0)
    #metric_der = metric_der.view(8, numsteps*numsteps)
    #metric_der = metric_der.transpose(0, 1)
    #metric_der = metric_der.view(numsteps*numsteps, 2, 4)
    #metric_der = metric_der.view(numsteps*numsteps, 2, 2, 2)
    return metric_der

In [ ]:
metric_der_not_using_decoder().shape

In [ ]:
metric_der.shape

#### the computation of metric derivatives to be fixed!! 

In [ ]:
torch.equal(metric_der_not_using_decoder(), metric_der)

In [ ]:
torch.max(metric_der_not_using_decoder() - metric_der)

In [ ]:
# compute the metric of the latent space and take it as initial condition
with torch.no_grad():
    metric = g(tgrid)
g_array = []
g_array.append(metric) # g_0 = metric
# we need to recompute metric_der and metric after every evo step
# let us compute evoluation of the metric
for i in range(steps):
    Ricci_tensor = Ric(tgrid, metric_inv, metric_der_not_using_decoder(tgrid,g_array[i]))
    g_new = g_array[i] - 2*Ricci_tensor*dt
    g_array.append(g_new)
    

In [ ]:
# it seems to blow up!
# probably there is an error in metric_der computation